# Data Preparation 
Using the Data we got, we have to make some modifications to use in our ABM
For example, we clean the routes to only have a single route between two edges

In [2]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
import random
from tqdm import tqdm, trange

data_path = '../' #set to wherever the data files are, will be used on every input

## Data Imports

In [41]:
route_blockages = pd.read_csv((data_path + 'route_blockage.csv'))

distances_df = pd.read_csv((data_path + 'distances.csv'))
distances = distances_df[["prev_port", "next_port", "distance"]]
distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes


prev_port      int64
next_port      int64
distance     float64
dtype: object

In [49]:
clean_distances = distances.sort_values(by=['prev_port', 'next_port', 'distance'])
clean_distances.drop_duplicates(subset=['prev_port', 'next_port'], keep='first', inplace=True, ignore_index=True)

In [53]:
# clean_distances.to_csv((data_path + 'clean_distances.csv'))

In [6]:
clean_distances = pd.read_csv((data_path + 'clean_distances.csv'))

In [51]:
G = nx.from_pandas_edgelist(clean_distances, "prev_port", "next_port", ["distance"], create_using=nx.Graph())


## Create Pruning Plans

When cutting the network, we want to ensure that we cut the correct nedges 

We therefore create a list indicating the number of edges, and then separate the file by which chokepoint (or all) is/are blocked


In [54]:

route_blockages_gib =  route_blockages[route_blockages['affected_by_gibraltar']==True]
route_blockages_mal =  route_blockages[route_blockages['affected_by_malacca']==True]
route_blockages_dov =  route_blockages[route_blockages['affected_by_dover']==True]
route_blockages_suez =  route_blockages[route_blockages['affected_by_suez']==True]
route_blockages_horm =  route_blockages[route_blockages['affected_by_hormuz']==True]
route_blockages_pan = route_blockages[route_blockages['affected_by_panama']==True]

route_blockages_horm = route_blockages_horm[["prev_port", "next_port"]]
route_blockages_gib =  route_blockages_gib[["prev_port", "next_port"]]
route_blockages_mal =  route_blockages_mal[["prev_port", "next_port"]]
route_blockages_dov =  route_blockages_dov[["prev_port", "next_port"]]
route_blockages_suez =  route_blockages_suez[["prev_port", "next_port"]]
route_blockages_pan = route_blockages_pan[["prev_port", "next_port"]]



In [55]:
route_blockages_total = pd.concat([route_blockages_horm, route_blockages_gib, route_blockages_mal, route_blockages_dov, route_blockages_suez, route_blockages_pan])
route_blockages_total = route_blockages_total.drop_duplicates()

In [56]:
"""
Commented out for your convenience, files are included
"""
# route_blockages_horm.to_csv((data_path + 'route_blockages_horm.csv'))
# route_blockages_mal.to_csv((data_path + 'route_blockages_mal.csv'))
# route_blockages_dov.to_csv((data_path + 'route_blockages_dov.csv'))
# route_blockages_suez.to_csv((data_path + 'route_blockages_suez.csv'))
# route_blockages_gib.to_csv((data_path + 'route_blockages_gib.csv'))
# route_blockages_pan.to_csv((data_path + 'route_blockages_pan.csv'))
# route_blockages_total.to_csv((data_path + 'route_blockages_total.csv'))

In [3]:
route_blockages_horm = pd.read_csv((data_path + 'route_blockages_horm.csv'))

## Pruning Functions

In [9]:
def Cut_Graph(G, route_blockages):
    print('Before Pruning',G.number_of_edges())
    # CRUCIAL: If the below line is not included, we prune both the additional as well as the ORIGINAL Graphs
    return_G = G.copy()
    for index in tqdm(range(len(route_blockages)), desc='Removing Edges', delay=0.5):
        try:
            return_G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
        except:
            pass
    print('\n', 'After Pruning',retunr_G.number_of_edges())
    return return_G

In [8]:
def Build_Cut_Graph(distances, route_blockages):
    G = nx.from_pandas_edgelist(distances, "prev_port", "next_port", ["distance"], create_using=nx.Graph())
    print('Before Pruning:','\n','Nodes: ',G.number_of_nodes(), 'Edges: ',G.number_of_edges())
    for index in tqdm(range(len(route_blockages)), desc='Removing Edges', delay=0.5):
        try:
            G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
        except:
            pass
    print('\n', 'After Pruning:','\n', 'Nodes: ',G.number_of_nodes(), 'Edges: ',G.number_of_edges())
    return G

In [19]:
G_without_Suez = Build_Cut_Graph(clean_distances, route_blockage_suez)

Before Pruning: 
 Nodes:  2912 Edges:  29838

 After Pruning: 
 Nodes:  2912 Edges:  29365
